# Multi-Party Computation Demo

Here is a demo to show how we can perform encypted operations using PySyft for Secure Multi-Party Computation (SMPC).

## Encryption Using Secure Multi-Party Computation

SMPC is at first glance a rather strange form of "encryption". Instead of using a public/private key to encrypt a variable, each value is split into multiple `shares`, each of which operate like a private key. Typically, these `shares` will be distributed amongst 2 or more _owners_. Thus, in order to decrypt the variable, all owners must agree to allow the decryption. In essence, everyone has a private key.

### Encrypt()

So, let's say we wanted to "encrypt" a variable `x`, we could do so in the following way.

 > Encryption doesn't use floats or real numbers but happens in a mathematical space called [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) which is basically the integers between `0` and `Q-1`, where `Q` is prime and "big enough" so that the space can contain all the numbers that we use in our experiments. In practice, given a value `x` integer, we do `x % Q` to fit in the ring. (That's why we avoid using number `x' > Q`).

In [1]:
import random

def encrypt(x):
    Q = 1234567891011
    share_a = random.randint(0, Q)
    share_b = random.randint(0, Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [2]:
encrypt(25)

(1029052497760, 878663863619, 561419420668)

As you can see here, we have split our variable `x` into 3 different shares, which could be sent to 3 different owners.

### Decrypt()

If we wanted to decrypt these 3 shares, we could simply sum them together and take the modulus of the result (mod Q)

In [3]:
def decrypt(*shares):
    Q = 1234567891011
    return sum(shares) % Q

In [4]:
a, b, c = encrypt(25)

In [5]:
decrypt(a, b, c)

25

In [6]:
d, e, f = encrypt(5)
decrypt(a + d, b + e, c + f)

30

In [7]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### Basic Encryption/Decryption

Encryption is as simple as taking any PySyft tensor and calling .share(). Decryption is as simple as calling .get() on the shared variable

In [8]:
x = torch.tensor([25])

In [9]:
encrypted_x = x.share(bob, alice, bill)

In [10]:
encrypted_x.get()

tensor([25])

### Introspecting the Encrypted Values

If we look closer at Bob, Alice, and Bill's workers, we can see the shares that get created!

In [11]:
x = torch.tensor([25]).share(bob, alice, bill)

In [12]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([1192794008])

In [13]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([664303720])

In [14]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([-1857097703])

In [15]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

As you can see, when we called `.share()` it simply split the value into 3 shares and sent one share to each of the parties! The truly extraordinary property of Secure Multi-Party Computation is the ability to perform computation **while the variables are still encrypted**.

# Encrypted Arithmetic

The API is constructed so that we can simply perform arithmetic like we would normal PyTorch tensors.

In [16]:
x = torch.tensor([25]).share(bob, alice)
y = torch.tensor([5]).share(bob, alice)

In [17]:
z = x + y
z.get()

tensor([30])

In [18]:
z = x - y
z.get()

tensor([20])

As it turns out, SMPC protocols exist which can allow this encrypted computation for the following operations:
- addition (which we've just seen)
- multiplication
- comparison

and using these basic underlying primitives, we can perform arbitrary computation!